# Introduction
Everyone who loves command line utilites will love this demo. And those that don't probably shouldn't be here.

## Fire up the cluster!!!!!!

### Start Zookeeper First.

In [1]:
zookeeper-server-start -daemon /Users/patrick/Documents/CP_Software/confluent-5.2.1/etc/kafka/zookeeper.properties

### Start some Kafka Brokers next.

In [2]:
kafka-server-start -daemon /Users/patrick/Documents/CP_Software/confluent-5.2.1/etc/kafka/server1.properties
kafka-server-start -daemon /Users/patrick/Documents/CP_Software/confluent-5.2.1/etc/kafka/server2.properties
kafka-server-start -daemon /Users/patrick/Documents/CP_Software/confluent-5.2.1/etc/kafka/server3.properties

### Are we running?

In [3]:
jps

5383 QuorumPeerMain
5432 SupportedKafka
5433 Jps
5418 SupportedKafka
5403 SupportedKafka


### Let's create a topic!

In [4]:
kafka-topics --bootstrap-server localhost:9092 --create --topic users \
    --replication-factor 3 \
    --partitions 3 \
    --config min.insync.replicas=2

### What topics are available?

In [5]:
kafka-topics --bootstrap-server localhost:9092 --list

users


### How is that topic defined?

In [6]:
kafka-topics --bootstrap-server localhost:9092 --topic users --describe

Topic:users	PartitionCount:3	ReplicationFactor:3	Configs:min.insync.replicas=2,segment.bytes=1073741824
	Topic: users	Partition: 0	Leader: 1	Replicas: 1,2,3	Isr: 1,2,3
	Topic: users	Partition: 1	Leader: 2	Replicas: 2,3,1	Isr: 2,3,1
	Topic: users	Partition: 2	Leader: 3	Replicas: 3,1,2	Isr: 3,1,2


### What's in that topic?

In [9]:
kafka-console-consumer --bootstrap-server localhost:9092 --topic users --from-beginning

{"registertime":1511299256550,"userid":"User_2","regionid":"Region_1","gender":"MALE"}
{"registertime":1505122567712,"userid":"User_5","regionid":"Region_1","gender":"MALE"}
{"registertime":1489777293550,"userid":"User_3","regionid":"Region_2","gender":"FEMALE"}
{"registertime":1500726224175,"userid":"User_3","regionid":"Region_5","gender":"OTHER"}
{"registertime":1516967717491,"userid":"User_7","regionid":"Region_6","gender":"OTHER"}
Processed a total of 5 messages


### How do we put messages in that topic?

In [8]:
ksql-datagen quickstart=users topic=users iterations=5 format=json

[2019-04-17 18:38:53,757] INFO AvroDataConfig values: 
	schemas.cache.config = 1
	enhanced.avro.schema.support = false
	connect.meta.data = true
 (io.confluent.connect.avro.AvroDataConfig:179)
User_5 --> ([ 1505122567712 | 'User_5' | 'Region_1' | 'MALE' ]) ts:1555540734136
User_3 --> ([ 1489777293550 | 'User_3' | 'Region_2' | 'FEMALE' ]) ts:1555540734168
User_2 --> ([ 1511299256550 | 'User_2' | 'Region_1' | 'MALE' ]) ts:1555540734520
User_3 --> ([ 1500726224175 | 'User_3' | 'Region_5' | 'OTHER' ]) ts:1555540734687
User_7 --> ([ 1516967717491 | 'User_7' | 'Region_6' | 'OTHER' ]) ts:1555540735127


### Is there anything else I can use?

In [11]:
kafkacat -L -b localhost:9092 -t users

Metadata for users (from broker -1: localhost:9092/bootstrap):
 3 brokers:
  broker 2 at 127.0.0.1:9093
  broker 3 at 127.0.0.1:9094 (controller)
  broker 1 at 127.0.0.1:9092
 1 topics:
  topic "users" with 3 partitions:
    partition 0, leader 1, replicas: 1,2,3, isrs: 1,2,3
    partition 1, leader 2, replicas: 2,3,1, isrs: 2,3,1
    partition 2, leader 3, replicas: 3,1,2, isrs: 3,1,2


### What about performance testing?

In [10]:
kafka-topics --bootstrap-server localhost:9092 --create --topic perf-test \
    --replication-factor 3 \
    --partitions 3 \
    --config min.insync.replicas=2

In [12]:
kafka-producer-perf-test --topic perf-test \
    --producer-props bootstrap.servers=localhost:9092 \
    --num-records 9999999 \
    --throughput -1 \
    --record-size 1500

70741 records sent, 14136.9 records/sec (20.22 MB/sec), 1107.1 ms avg latency, 1735.0 ms max latency.
90890 records sent, 18170.7 records/sec (25.99 MB/sec), 1086.2 ms avg latency, 1788.0 ms max latency.
87370 records sent, 17474.0 records/sec (25.00 MB/sec), 1182.6 ms avg latency, 2016.0 ms max latency.
139960 records sent, 27992.0 records/sec (40.04 MB/sec), 798.9 ms avg latency, 1864.0 ms max latency.
271660 records sent, 54332.0 records/sec (77.72 MB/sec), 380.9 ms avg latency, 868.0 ms max latency.
263460 records sent, 52692.0 records/sec (75.38 MB/sec), 389.0 ms avg latency, 753.0 ms max latency.
241962 records sent, 48392.4 records/sec (69.23 MB/sec), 419.6 ms avg latency, 708.0 ms max latency.


### Ok, that wasn't bad but can we do better?

In [13]:
kafka-producer-perf-test --topic perf-test \
    --producer-props bootstrap.servers=localhost:9092 batch.size=150000 \
    --num-records 9999999 \
    --throughput -1 \
    --record-size 1500

478600 records sent, 95700.9 records/sec (136.90 MB/sec), 205.5 ms avg latency, 426.0 ms max latency.
628397 records sent, 125679.4 records/sec (179.79 MB/sec), 170.4 ms avg latency, 341.0 ms max latency.
731313 records sent, 146204.1 records/sec (209.15 MB/sec), 149.5 ms avg latency, 295.0 ms max latency.
753489 records sent, 150697.8 records/sec (215.57 MB/sec), 145.3 ms avg latency, 277.0 ms max latency.
752667 records sent, 150533.4 records/sec (215.34 MB/sec), 145.6 ms avg latency, 268.0 ms max latency.
757251 records sent, 151450.2 records/sec (216.65 MB/sec), 143.9 ms avg latency, 264.0 ms max latency.
733979 records sent, 146766.4 records/sec (209.95 MB/sec), 149.2 ms avg latency, 307.0 ms max latency.
747942 records sent, 149409.1 records/sec (213.73 MB/sec), 145.9 ms avg latency, 268.0 ms max latency.
747973 records sent, 149564.7 records/sec (213.95 MB/sec), 146.5 ms avg latency, 264.0 ms max latency.
765476 records sent, 153095.2 records/sec (219.00 MB/sec), 142.8 ms avg la

### Let's blow the roof off of this thing!

In [14]:
kafka-producer-perf-test --topic perf-test \
    --producer-props bootstrap.servers=localhost:9092 batch.size=150000 compression.type=lz4 \
    --num-records 9999999 \
    --throughput -1 \
    --record-size 1500

1641684 records sent, 328205.5 records/sec (469.50 MB/sec), 6.8 ms avg latency, 261.0 ms max latency.
2351824 records sent, 470270.7 records/sec (672.73 MB/sec), 5.0 ms avg latency, 36.0 ms max latency.
2376335 records sent, 475172.0 records/sec (679.74 MB/sec), 3.2 ms avg latency, 20.0 ms max latency.
2360036 records sent, 472007.2 records/sec (675.21 MB/sec), 3.1 ms avg latency, 16.0 ms max latency.
9999999 records sent, 439637.694540 records/sec (628.91 MB/sec), 4.30 ms avg latency, 261.00 ms max latency, 3 ms 50th, 11 ms 95th, 21 ms 99th, 36 ms 99.9th.


### Clean up this mess.

In [15]:
kafka-server-stop

In [16]:
zookeeper-server-stop